In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/home/lyz/hf-models/Qwen2.5-3B-Instruct/"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

2025-07-08 11:44:47.760174: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
response

'Certainly! A large language model (LLM) is a type of artificial intelligence designed to understand and generate human-like text based on vast amounts of training data. These models are trained using machine learning algorithms to recognize patterns and relationships within the text they encounter during training. \n\nKey characteristics of large language models include:\n\n1. **Size**: They are typically trained on enormous datasets, often consisting of billions or trillions of words.\n2. **Complexity**: They have complex architectures that allow them to handle various types of language tasks, including translation, summarization, question-answering, and even creative writing.\n3. **Interactivity**: They can interact with users in real-time, responding to queries and generating text dynamically.\n4. **Adaptability**: They can be fine-tuned for specific domains or applications, making them highly versatile.\n\nLarge language models like myself are capable of understanding context, rea

In [4]:
PROMPT_TEMPLATE = """你是一个专业的植物命名实体识别专家，请识别句子中实体：@{0}@


其中需要识别的实体如下：
- 植物 (plant)：植物的通用名称或俗称，如细瘦糯米条/木里秋葵/察隅冷杉。
- 保护等级 (class)：植物受到的保护级别，如国家级、省级、濒危、二级等
- 生长环境 (environment)：描述植物生长的自然环境、土壤条件或生态位，如沼泽/冻原/平原/芦苇丛/季雨林/峡谷/苔藓地。
- 分布地区 (area)：植物在地理上的分布区域，如巴西/中国/印度/阿富汗。
- 海拔 (altitude)：植物主要生长的海拔范围，如海拔4200米。

识别的实体需要来自原始句子，并且是连续的单词，如果存在多个实体则存为列表。识别结果填充为如下json，不需要有其他任何输出
{
    "plant": [],
    "class": [],
    "environment": [],
    "area": [],
    "altitude": []
}
"""

In [5]:
import json
for line in open("test.jsonl").readlines():
    origin_text = json.loads(line)["text"]
    result_label = {}

    try:
        messages = [
            {"role": "user", "content": PROMPT_TEMPLATE.replace("@{0}@", origin_text)}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
        
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        
        result = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        try:
            result = json.loads(result)
        except:
            result = {}
    
        for category in result.keys():
            if len(result[category]) > 0:
                for word in result[category]:
                    if word in origin_text:
                        if category not in result_label:
                            result_label[category] = {}
    
                        result_label[category][word] = [[origin_text.index(word), origin_text.index(word)+len(word)-1]]
    except:
        pass

    with open("submit.jsonl", "a") as up:
        up.write(json.dumps({
            "text": origin_text,
            "label": result_label
        }, ensure_ascii=False) + "\n")

In [1]:

origin_text

NameError: name 'origin_text' is not defined

In [49]:
word

'苔藓地'